# PROMETEO_v1.0

## Paquetería

In [37]:
import ast
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import shutil
import textwrap
import tiktoken
import umap.umap_ as umap
from dotenv import load_dotenv, find_dotenv
from langchain import PromptTemplate
from langchain.document_loaders import DirectoryLoader, PyPDFLoader
from langchain_core.runnables import RunnableParallel
from langchain_core.runnables import RunnablePassthrough
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.output_parsers import StrOutputParser
from sklearn.mixture import GaussianMixture
from typing import Optional

## Funciones

In [38]:
def num_tokens_from_string(string: str) -> int:
    """Cuenta el número de tokens en el documento
    proporcionado."""
    encoding = tiktoken.get_encoding("cl100k_base")
    num_tokens = len(encoding.encode(string))
    return num_tokens

def reduce_cluster_embeddings(
    embeddings: np.ndarray,
    dim: int,
    n_neighbors: Optional[int] = None,
    metric: str = "cosine",
) -> np.ndarray:
    """Esta función no la comprendo. Estudie
    para entenderla."""
    if n_neighbors is None:
        n_neighbors = int((len(embeddings) - 1) ** 0.5)
    return umap.UMAP(
        n_neighbors=n_neighbors, n_components=dim, metric=metric
    ).fit_transform(embeddings)

def get_optimal_clusters(embeddings: np.ndarray, max_clusters: int = 50, random_state: int = 1234):
    """Obtiene el número óptimo de clústers."""
    max_clusters = min(max_clusters, len(embeddings))
    bics = [GaussianMixture(n_components=n, random_state=random_state).fit(embeddings).bic(embeddings)
            for n in range(1, max_clusters)]
    return np.argmin(bics) + 1

def gmm_clustering(embeddings: np.ndarray, threshold: float, random_state: int = 0):
    """Clusteriza con el método GMM."""
    n_clusters = get_optimal_clusters(embeddings)
    gm = GaussianMixture(n_components=n_clusters, random_state=random_state).fit(embeddings)
    probs = gm.predict_proba(embeddings)
    labels = [np.where(prob > threshold)[0] for prob in probs]
    return labels, n_clusters

def format_cluster_texts(df):
    """Agrupa los textos de cada clúster el listas."""
    clustered_texts = {}
    for cluster in df['Cluster'].unique():
        cluster_texts = df[df['Cluster'] == cluster]['Texto'].tolist()
        clustered_texts[cluster] = " --- ".join(cluster_texts)
    return clustered_texts

def wrap_text_preserve_newlines(text, width=80):
    """Formato para respuestas."""
    lines = text.split('\n')
    wrapped_lines = [textwrap.fill(line, width=width) for line in lines]
    wrapped_text = '\n'.join(wrapped_lines)
    return wrapped_text

def process_llm_response(llm_response):
    """Generador de referencias."""
    print(wrap_text_preserve_newlines(llm_response['answer']))
    print('\nReferencias:')
    for contexto in llm_response["context"][:5]:
        print(contexto)

## Parámetros

In [39]:
load_dotenv(find_dotenv())
embeddings = OpenAIEmbeddings(
)

detailed_turbo_llm = turbo_llm = ChatOpenAI(
    temperature=0,
    model_name='gpt-3.5-turbo-0125'
)

template = """Tu tarea como revisor bibliográfico profesional
es crear resúmenes extremadamente detallados del siguiente
texto: {text} """

prompt = PromptTemplate.from_template(template)
chain = prompt | detailed_turbo_llm | StrOutputParser()

turbo_llm = ChatOpenAI(
    temperature=0.7,
    model_name='gpt-3.5-turbo-0125'
)

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=2000,
    chunk_overlap=100
)

## Carga de documentos

In [40]:
# Asegúrate de que haya PDFs en la carpeta 'docs'
documents = DirectoryLoader('./docs/', glob="./*.pdf", loader_cls=PyPDFLoader).load()
# Tratameinto de caracteres indeseados
for d in documents:
    d.page_content = d.page_content.replace('\n', ' ').replace('\t', ' ')

docs = text_splitter.split_documents(documents)
texts = [doc.page_content for doc in docs]

d_sorted = sorted(docs, key=lambda x: x.metadata["source"])
d_reversed = list(reversed(d_sorted))
concatenated_content = "\n\n\n --- \n\n\n".join(
    [doc.page_content for doc in d_reversed]
)
print(
    "Número de tokens en el documento: %s"
    % num_tokens_from_string(concatenated_content)
)

Número de tokens en el documento: 15416


## ¿Quieres crear o cargar una incrustación?

In [42]:
user_input = input("¿Crear (1) o cargar (2) incrustación?: ")
destino_emb = r'c:\Users\luisr\OneDrive\Documentos\GitHub\1day_1thing\prometeo\embed'

if user_input.lower() == "1":
    print(f'Elegiste crear de cero una incrustación.\n')
    global_embeddings = [embeddings.embed_query(txt) for txt in texts]

    embed_name = input('¿Cómo se llama esta incrustación?: ') + '_emb' + '.txt'
    emb = rf'c:\Users\luisr\OneDrive\Documentos\GitHub\1day_1thing\prometeo\{embed_name}'
    with open(rf'./{embed_name}', 'w') as f:
        for i in global_embeddings:
            f.write("%s\n" % i)
    print(f'Estás usando la incrustación: {embed_name}')
    shutil.move(emb, destino_emb)
    
elif user_input.lower() == "2":
    print('Elegiste cargar una incrustación ya creada.\n')
    global_embeddings = []

    embed_name = input('Nombre de la incrustación: ') + '_emb' + '.txt'
    print(f'Estás usando la incrustación: {embed_name}')
    with open(rf'./embed/{embed_name}', 'r') as f:
        for i in f:
            x = ast.literal_eval(i.strip())  # Convertir la cadena a lista de números
            global_embeddings.append(x)

    global_embeddings = np.array(global_embeddings, dtype=float)
    
elif user_input != "1" and user_input != "2":
    print('No seleccionaste ninguna incrustación.\n')



Elegiste crear de cero una incrustación.

Estás usando la incrustación: meto_3_emb.txt


## Procesamiento de texto

In [43]:
dim = 2
global_embeddings_reduced = reduce_cluster_embeddings(global_embeddings, dim)
labels, _ = gmm_clustering(global_embeddings_reduced, threshold=0.5)
simple_labels = [label[0] if len(label) > 0 else -1 for label in labels]

df = pd.DataFrame({
    'Texto': texts,
    'Embedding': list(global_embeddings_reduced),
    'Cluster': simple_labels
})

clustered_texts = format_cluster_texts(df)
summaries = {}
for cluster, text in clustered_texts.items():
    summary = chain.invoke({"text": text})
    summaries[cluster] = summary
embedded_summaries = [embeddings.embed_query(summary) for summary in summaries.values()]
embedded_summaries_np = np.array(embedded_summaries)
labels, _ = gmm_clustering(embedded_summaries_np, threshold=0.5)
simple_labels = [label[0] if len(label) > 0 else -1 for label in labels]
clustered_summaries = {}
for i, label in enumerate(simple_labels):
    if label not in clustered_summaries:
        clustered_summaries[label] = []
    clustered_summaries[label].append(list(summaries.values())[i])
final_summaries = {}
for cluster, texts in clustered_summaries.items():
    combined_text = ' '.join(texts)
    summary = chain.invoke({"text": combined_text})
    final_summaries[cluster] = summary
texts_from_df = df['Texto'].tolist()
texts_from_clustered_texts = list(clustered_texts.values())
texts_from_final_summaries = list(final_summaries.values())

combined_texts = texts_from_df + texts_from_clustered_texts + texts_from_final_summaries

## ¿Quieres crear o cargar texto procesado? (hacer)

In [44]:
# Nombre del archivo de texto
file_name = input('Nombre del archivo: ') + '.txt'

# Escribir la lista en el archivo de texto
with open(file_name, 'w', encoding='utf-8') as f:
    for t in combined_texts:
        f.write("%s\n" % t)

# Leer el contenido del archivo y mostrarlo
with open(file_name, 'r', encoding='utf-8') as f:
    content = f.read()

textos = text_splitter.split_text(content)

# Guardar los kb en una carpeta (kbs) (hacer)

### 1. Enseñar

In [45]:
destino_kb = r'c:\Users\luisr\OneDrive\Documentos\GitHub\1day_1thing\prometeo\kbs'
persist_directory = input('¿Cómo se llamará este knowledge-base?: ') + '_kb'
vectorstore = Chroma.from_texts(texts=textos,
                                embedding=embeddings,
                                persist_directory=persist_directory)
vectorstore.persist()
vectorstore = None
os.system(f'zip -r db.zip ./{persist_directory}')

1

### 2. Recordar (si y solo si Enseñar=Done) y pregunta de control

In [46]:
persist_directory = input('¿Qué knoledge-base deseas usar?: ') + '_kb'
embedding = OpenAIEmbeddings()
vectorstore = Chroma(persist_directory=persist_directory, 
                  embedding_function=embedding)

def adjust_final_number(string: str, max_threshold: int, initial_number: int) -> int:
    final_number = initial_number
    while final_number < max_threshold:
        retriever = vectorstore.as_retriever(search_kwargs={"k": final_number})
        docs = retriever.invoke(string)
        text = "".join([doc.page_content for doc in docs])
        if num_tokens_from_string(text) < max_threshold:
            final_number += 1
        else:
            break
    return final_number

final_number = adjust_final_number("¿Cuál es el tema principal del documento?", 10000, 4)
print(f'K final es: {final_number}\n')
retriever = vectorstore.as_retriever(search_kwargs={"k": final_number})

K final es: 35



In [47]:
# Se personaliza el LLM como Prometeo
template = """
Eres Prometeo, un asistente personal de revisión biliográfica que habla Español.

Tu tarea consiste en:

1. Leer detalladamente la información proporcionada en documentos que generalmente son
artículos científicos en formato PDF.

2. Proporcionar respuestas extremadamente detalladas a cualquier tipo de pregunta relacionada 
con el contexto, especialmente a preguntas que involucren peticiones. Evita concluir tus respuestas con un 
párrafo que comience con 'En resumen,...'. Varía las conclusiones de tus respuestas para que 
sean más diversas y creativas.

3. Ser carismático y ofrecer información sobre ti y tus funciones.

SIEMPRE debes responder en Español.

SIEMPRE debes responder con base al siguiente contexto: {context}.

NUNCA hables específicamente del contexto.

NUNCA terminar las respuestas con un resumen o una conclusión predecible.

Teniendo en cuenta TODO lo anterior, responde la siguiente pregunta: {question}
"""

prometeo_prompt = PromptTemplate(
    template=template, input_variables=["context", "question"]
)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    RunnablePassthrough.assign(context=(lambda x: format_docs(x["context"])))
    | prometeo_prompt
    | turbo_llm
    | StrOutputParser()
)

rag_chain_with_source = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
).assign(answer=rag_chain
)

In [48]:
# Demo
query = input("Hazme una pregunta: ")
print(query)
llm_response = rag_chain_with_source.invoke(query)
process_llm_response(llm_response)

¿Cuál es la idea principal de investigación?
La idea principal de investigación es explorar y desarrollar un marco integral
de innovación en modelos de negocio para el emprendimiento tecnológico,
centrándose en la relación entre los modelos de negocio, la innovación y el
rendimiento empresarial. Se busca identificar cómo diseñar y gestionar modelos
de negocio de manera efectiva para crear valor sostenible y mantener la ventaja
competitiva en un mercado en constante evolución. Además, se pretende abordar
las complejidades y desafíos asociados con la innovación en modelos de negocio,
especialmente en el contexto de la tecnología y el emprendimiento.

Referencias:
page_content='En resumen, la revisión bibliográfica detallada aborda una amplia gama de temas relacionados con la innovación en modelos de negocio y el emprendimiento tecnológico. Se destacan la importancia de la estrategia, la creación de valor, la innovación y la adaptación a los cambios en el entorno empresarial para lograr e

In [49]:
# Demo
query = input("Hazme una pregunta: ")
print(query)
llm_response = rag_chain_with_source.invoke(query)
process_llm_response(llm_response)

¿Cuál es el objetivo u objetivos de investigación?
El objetivo de la investigación es establecer el Modelo de Negocio Innovador
para el Emprendimiento Tecnológico como un nuevo constructo único, a través de
una revisión detallada de la literatura. Se busca identificar los parámetros
necesarios para realizar una revisión sistemática posterior, así como establecer
un marco integral que permita la creación y gestión de modelos de negocio para
el emprendimiento tecnológico. Además, se pretende analizar la calidad de los
artículos seleccionados y extraer datos relevantes para comprender la relación
entre los modelos de negocio, la innovación y la sostenibilidad económica en el
contexto del emprendimiento tecnológico.

Referencias:
page_content='En resumen, la revisión bibliográfica detallada aborda una amplia gama de temas relacionados con la innovación en modelos de negocio y el emprendimiento tecnológico. Se destacan la importancia de la estrategia, la creación de valor, la innovación y l

In [50]:
# Demo
query = input("Hazme una pregunta: ")
print(query)
llm_response = rag_chain_with_source.invoke(query)
process_llm_response(llm_response)

¿Explícame el Modelo de Negocio Innovador para el Emprendimiento Tecnológico?
El Modelo de Negocio Innovador para el Emprendimiento Tecnológico se basa en la
creación y captura de valor a través de la capitalización y comercialización de
tecnologías innovadoras. Este modelo se enfoca en la experimentación, el diseño,
la generación, la renovación y la implementación de procesos que permitan
integrar elementos clave como la migración de valor, evaluación de oportunidades
y riesgos, capacidades dinámicas, redes de interés y estrategias
institucionales. Además, se centra en la creación de valor para los clientes, la
configuración de propuestas de valor, la captura de valor y la creación y
entrega de valor de manera efectiva.

Para tener éxito en el emprendimiento tecnológico, es fundamental contar con
recursos organizativos y capacidades que permitan la creación y captura de
valor. La colaboración con diferentes actores y la construcción de una red de
negocios sólida también son aspectos c

In [51]:
# Demo
query = input("Hazme una pregunta: ")
print(query)
llm_response = rag_chain_with_source.invoke(query)
process_llm_response(llm_response)

¿Cuál es la metodología de la investigación?
La metodología de la investigación se basa en un enfoque cualitativo para
analizar los datos recopilados de la literatura existente. Se llevó a cabo un
proceso de revisión sistemática porque este enfoque se utiliza para identificar,
evaluar y sintetizar la literatura disponible de manera comprensiva, explícita y
reproducible. Además, se incluye la búsqueda sistemática de la literatura. Para
garantizar la calidad de la investigación, se documentaron los hallazgos de la
literatura, la selección de palabras clave y la evaluación de los resultados.
Los datos fueron extraídos por dos revisores independientes y cualquier
desacuerdo entre los revisores se discutió y se llegó a un consenso. Además, se
identificaron manualmente los artículos duplicados y se eliminaron. En caso de
que no se accediera a un artículo de texto completo, se intentó contactar a los
autores y, si no se recibía respuesta, el artículo se excluía del estudio. Los
artículos para

In [52]:
# Demo
query = input("Hazme una pregunta: ")
print(query)
llm_response = rag_chain_with_source.invoke(query)
process_llm_response(llm_response)

¿Cuáles fueron los resultados más importantes de la investigación?
Los resultados más importantes de la investigación se centran en la
identificación de los elementos clave para la innovación en modelos de negocio
en el contexto del emprendimiento tecnológico. Se destaca la importancia de la
estrategia, la creación de valor, la innovación y la adaptación a los cambios en
el entorno empresarial para lograr el éxito en este campo. Asimismo, se resalta
la necesidad de evaluar de manera independiente el impacto de diferentes
variables en el rendimiento empresarial, así como la importancia de considerar
la comercialización de innovaciones tecnológicas a través de la digitalización y
el marketing en línea. Estos hallazgos sugieren áreas de investigación futuras
para explorar en mayor profundidad la relación entre los modelos de negocio, la
innovación y el rendimiento empresarial en el contexto del emprendimiento
tecnológico.

Referencias:
page_content='En resumen, la revisión bibliográfica d

In [55]:
# Demo
query = input("Hazme una pregunta: ")
print(query)
llm_response = rag_chain_with_source.invoke(query)
process_llm_response(llm_response)

¿Cuáles son los elementos clave para la innovación en modelos de negocio en el contexto del emprendimiento tecnológico?
Los elementos clave para la innovación en modelos de negocio en el contexto del
emprendimiento tecnológico incluyen la evaluación de oportunidades y riesgos, la
capacidad dinámica de la empresa, el establecimiento de redes con los
stakeholders, el uso eficiente de los recursos, la consideración de la ontología
institucional, y la orientación hacia la sostenibilidad económica. Estos
elementos son fundamentales para diseñar, implementar y modificar modelos de
negocio de manera efectiva en el ámbito del emprendimiento tecnológico,
garantizando así el éxito y la creación de valor sostenible.

Referencias:
page_content='En resumen, la revisión bibliográfica detallada aborda una amplia gama de temas relacionados con la innovación en modelos de negocio y el emprendimiento tecnológico. Se destacan la importancia de la estrategia, la creación de valor, la innovación y la adapt

In [57]:
# Demo
query = input("Hazme una pregunta: ")
print(query)
llm_response = rag_chain_with_source.invoke(query)
process_llm_response(llm_response)

¿Cuáles son y en qué consisten los elementos que son clave para la innovación en modelos de negocio en el contexto del emprendimiento tecnológico?
En el contexto del emprendimiento tecnológico, los elementos clave para la
innovación en modelos de negocio incluyen la propuesta de valor, la red de
valor, la captura de valor y la creación y entrega de valor. Estos elementos son
fundamentales para el éxito en la innovación de modelos de negocio en el ámbito
tecnológico. La propuesta de valor se refiere a la oferta única y atractiva que
la empresa proporciona a sus clientes, la red de valor se refiere a las
relaciones con los socios y clientes que generan valor conjunto, la captura de
valor se refiere a cómo la empresa monetiza su propuesta de valor y la creación
y entrega de valor se refiere a cómo la empresa desarrolla y ofrece su propuesta
de valor de manera efectiva. Estos elementos son cruciales para diseñar y
gestionar modelos de negocio innovadores que generen valor sostenible y
mant

In [ ]:
# Demo
query = input("Hazme una pregunta: ")
print(query)
llm_response = rag_chain_with_source.invoke(query)
process_llm_response(llm_response)

In [ ]:
# Demo
query = input("Hazme una pregunta: ")
print(query)
llm_response = rag_chain_with_source.invoke(query)
process_llm_response(llm_response)

In [ ]:
# Demo
query = input("Hazme una pregunta: ")
print(query)
llm_response = rag_chain_with_source.invoke(query)
process_llm_response(llm_response)

In [ ]:
# Demo
query = input("Hazme una pregunta: ")
print(query)
llm_response = rag_chain_with_source.invoke(query)
process_llm_response(llm_response)

# Hacer historial y guardar conversaciones para medir efectividad.